In [2]:
import pickle
import faiss
'''
THESE LIBRARIES ARE NOT WORKING FOR ME 
I CANT SPEND ALL MY TIME FIXING DEPENDENCIES
'''
import annoy 
import nmslib

Your CPU supports instructions that this binary was not compiled to use: AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [3]:
def load_data():
    with open('/Users/austinwilson/Downloads/movies.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
vectors = data["vector"]
names = data["name"]
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.01780608, -0.14265831,  0.10308606, ...,  0.09659795,
         -0.17529577, -0.03061521],
        [-0.03357764,  0.16418771,  0.21801303, ...,  0.16502103,
         -0.09166156,  0.05047869],
        [-0.2761452 , -0.01991325, -0.04969981, ...,  0.0258275 ,
         -0.08328608, -0.0152858 ],
        ...,
        [ 0.05142734, -0.01683608, -0.20441587, ...,  0.00045828,
          0.14679626,  0.2462584 ],
        [ 0.04491899, -0.02819411, -0.09472758, ..., -0.02152078,
          0.16223577,  0.19897607],
        [ 0.02531924,  0.03099714,  0.06437534, ..., -0.07260127,
          0.0467432 ,  0.07893164]], dtype=float32)}

# a) LSH 

In [24]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [25]:
index = LSHIndex(data["vector"], data["name"])
index.build()

### movies similar to GoldenEye

In [29]:
index.query(data['vector'][1:2])


['Days of Thunder (1990)',
 'GoldenEye (1995)',
 'Executive Decision (1996)',
 'Groundhog Day (1993)',
 'Dead Man Walking (1995)',
 'Star Trek: The Wrath of Khan (1982)',
 'Batman Returns (1992)',
 'Alien (1979)',
 'Time to Kill, A (1996)',
 'Sneakers (1992)']

# b) Exhaustive Search

In [17]:
class ExactIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [18]:
index = ExactIndex(data["vector"], data["name"])
index.build()

### look for movies that are most similar to GoldenEye

In [22]:
index.query(data['vector'][1:2])


['GoldenEye (1995)',
 'Under Siege (1992)',
 'Die Hard: With a Vengeance (1995)',
 'Demolition Man (1993)',
 'Batman Returns (1992)',
 'True Lies (1994)',
 'Cliffhanger (1993)',
 'Bad Boys (1995)',
 'Under Siege 2: Dark Territory (1995)',
 'Perfect World, A (1993)']

# c) product quantization

In [35]:
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                        self.dimension, 
                                        number_of_partition, 
                                        search_in_x_partitions, 
                                        subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

### movies most similar to Toy Story 

In [36]:
index = IVPQIndex(data["vector"], data["name"])
index.build()

In [37]:
index.query(data['vector'][0:1])

['Toy Story (1995)',
 'Fifth Element, The (1997)',
 'Men in Black (1997)',
 'James and the Giant Peach (1996)',
 'Phenomenon (1996)',
 'Hunchback of Notre Dame, The (1996)',
 'Star Trek: First Contact (1996)',
 'Twister (1996)',
 'Willy Wonka and the Chocolate Factory (1971)',
 "My Best Friend's Wedding (1997)"]

# d) trees and graphs

In [13]:
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimension)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(
              vector.tolist(), 
              k, 
              search_k=64)#search_in_x_trees)                                           
        return [self.labels[i] for i in indices]

In [14]:
index = AnnoyIndex(data["vector"], data["name"])
index.build()

/var/folders/jw/zs3wtf3n0vb9s135vkm2wn700000gn/T/ipykernel_29130/2669460728.py:8: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  self.index = annoy.AnnoyIndex(self.dimension)


In [20]:
index.query(data['vector'][5])

['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Hate (Haine, La) (1995)',
 'Ridicule (1996)',
 'White Balloon, The (1995)',
 "I Can't Sleep (J'ai pas sommeil) (1994)",
 'To Live (Huozhe) (1994)',
 'Faust (1994)',
 "Star Maker, The (Uomo delle stelle, L') (1995)",
 'Savage Nights (Nuits fauves, Les) (1992)',
 'Breaking the Waves (1996)']

# e) HNSW

In [16]:
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
            self.index = nmslib.init(method='hnsw', space='cosinesimil')
            self.index.addDataPointBatch(self.vectors)
            self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [17]:
index = NMSLIBIndex(data["vector"], data["name"])
index.build()

In [21]:
index.query(data['vector'][1])

['GoldenEye (1995)',
 'Under Siege (1992)',
 'Die Hard: With a Vengeance (1995)',
 'True Lies (1994)',
 'Demolition Man (1993)',
 'Batman Returns (1992)',
 'Die Hard (1988)',
 'Cliffhanger (1993)',
 'Batman (1989)',
 'Hard Target (1993)']

In [3]:
faiss.MatrixStats(vectors).comments.split("\n")


['analyzing 1682 vectors of size 64',
 'no NaN or Infs in data',
 'all vectors are distinct',
 'range of L2 norms=[0.747558, 1.80436] (0 null vectors)',
 'matrix contains no 0s',
 'no constant dimensions',
 'no dimension has a too large mean',
 'stddevs per dimension are in [0.112036 0.158214]',
 '']

In [4]:

index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

In [5]:
search_vector = vectors[90:91]
distances, indices = index.search(search_vector, 10)

In [6]:
print(f"The most similar movies to {names[90]} are:\n")
print([names[i] for i in indices[0]])

The most similar movies to Nightmare Before Christmas, The (1993) are:

['Nightmare Before Christmas, The (1993)', 'Heavy Metal (1981)', 'Sirens (1994)', 'Beauty and the Beast (1991)', 'Akira (1988)', 'Fantasia (1940)', 'Benny & Joon (1993)', 'Barbarella (1968)', "Pete's Dragon (1977)", 'James and the Giant Peach (1996)']


In [7]:
# LSH
quantizer = faiss.IndexFlatL2(vectors.shape[1])
index = faiss.IndexIVFPQ(quantizer, 
                         vectors.shape[1], 
                         100,             # number_of_partition,
                         8,               # search_in_x_partitions, 
                         8)               # subvector_size
index.train(vectors)
index.add(vectors)
search_vector = vectors[90:91]
distances, indices = index.search(search_vector, 10)
print(f"The most similar movies to {names[90]} are:\n")
print([names[i] for i in indices[0]])

The most similar movies to Nightmare Before Christmas, The (1993) are:

['Nightmare Before Christmas, The (1993)', 'Heavy Metal (1981)', 'Pink Floyd - The Wall (1982)', 'Akira (1988)', 'Hamlet (1996)', 'Full Metal Jacket (1987)', 'Supercop (1992)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)']


In [19]:
# LSH
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [20]:
index = LSHIndex(data["vector"], data["name"])
index.build()

In [21]:
index.query(data['vector'][0])

ValueError: not enough values to unpack (expected 2, got 1)